# Sheet

# Introducing DuckDB

DuckDB provides a built‑in function, read_csv_auto, which lets you query CSV files on‑the‑fly without any pre‑importing. You can also leverage DuckDB to read on-the-fly other file types such as JSON or parquet, using `read_parquet`, and `read_json` functions respectively. Using these functions, yo don't need to load the data into your tables.

The example below shows usage of the `read_csv` function.

In [40]:
-- Drop the table if it already exists to avoid errors
DROP TABLE IF EXISTS sample_data;

-- Create a new table by reading the CSV file directly using read_csv_auto
CREATE TABLE sample_data AS
SELECT *
FROM read_csv_auto('sample.csv');

-- Select all data from the table to see the contents
SELECT * FROM sample_data;

id 
 name 
 age 
 city 
 
 
 
 
 0 
 1 
 Alice 
 30 
 New York 
 
 
 1 
 2 
 Bob 
 25 
 Los Angeles 
 
 
 2 
 3 
 Charlie 
 35 
 Chicago 
 
 
 3 
 4 
 Dave 
 28 
 Houston

## Columnar optimizations

In DuckDB you can also use functions such as [`approx_count_distinct`](https://duckdb.org/docs/sql/functions/aggregates.html#approximate-aggregates) that benefit from DuckDB's columnar optimizations. These functions can efficiently estimate unique counts over large datasets.

Check [this document](https://duckdb.org/docs/sql/functions/overview) for an overview of other DuckDB functions built for efficient querying of your data.

In [42]:
-- Drop the table if it already exists
DROP TABLE IF EXISTS sample_numbers;

-- Create a table by reading the large CSV file using read_csv_auto
CREATE TABLE sample_numbers AS
SELECT *
FROM read_csv_auto('data.csv');

-- Compute the exact count of distinct values
SELECT COUNT(DISTINCT value) AS exact_distinct_values
FROM sample_numbers;

-- Compute the approximate count of distinct values using approx_count_distinct
SELECT approx_count_distinct(value) AS approx_distinct_values
FROM sample_numbers;

approx_distinct_values 
 
 
 
 
 0 
 11

## Advanced Analytics with Window Functions

DuckDB also supports advanced SQL analytics, including window functions. For example, you can rank rows based on a specific metric

In [29]:
-- Drop the table if it already exists
DROP TABLE IF EXISTS my_table;

-- Create a sample table with columns "name" and "value"
CREATE TABLE my_table (
    name VARCHAR,
    value INTEGER
);

-- Insert sample data into the table
INSERT INTO my_table (name, value)
VALUES
    ('Alice', 50),
    ('Bob', 70),
    ('Charlie', 60),
    ('Dave', 90);

-- Use a window function to assign a rank based on the "value" column (highest value gets rank 1)
SELECT 
    name,
    value,
    row_number() OVER (ORDER BY value DESC) AS rank
FROM my_table;


name 
 value 
 rank 
 
 
 
 
 0 
 Dave 
 90 
 1 
 
 
 1 
 Bob 
 70 
 2 
 
 
 2 
 Charlie 
 60 
 3 
 
 
 3 
 Alice 
 50 
 4

In [30]:
-- --------------------------------------------------
-- Oil Prices by Geo: Creating a table and computing statistics
-- --------------------------------------------------

-- Drop the table if it already exists to avoid conflicts
DROP TABLE IF EXISTS oil_prices;

-- Create a new table for oil prices by geographic region
CREATE TABLE oil_prices (
    id INTEGER,
    geo VARCHAR,
    price DOUBLE
);

-- Insert sample data into the oil_prices table
INSERT INTO oil_prices VALUES 
(1, 'USA', 70.5),
(2, 'USA', 68.2),
(3, 'Canada', 72.1),
(4, 'Russia', 65.0),
(5, 'Saudi Arabia', 67.8),
(6, 'Iran', NULL),
(7, 'Venezuela', 60.0),
(8, 'Norway', 74.0),
(9, 'Mexico', 69.5),
(10, 'USA', 71.2),
(11, 'Nigeria', 80.0),
(12, 'Nigeria', 82.3),
(13, 'Brazil', 64.5),
(14, 'Brazil', 63.8),
(15, 'USA', 120.0);

-- Describe the schema of the oil_prices table
DESCRIBE oil_prices;

-- Compute summary statistics for the oil_prices table
SUMMARIZE oil_prices;



column_name 
 column_type 
 min 
 max 
 approx_unique 
 avg 
 std 
 q25 
 q50 
 q75 
 count 
 null_percentage 
 
 
 
 
 0 
 id 
 INTEGER 
 1 
 15 
 17 
 8.0 
 4.47213595499958 
 4 
 8 
 12 
 15 
 0.00 
 
 
 1 
 geo 
 VARCHAR 
 Brazil 
 Venezuela 
 11 
 None 
 None 
 None 
 None 
 None 
 15 
 0.00 
 
 
 2 
 price 
 DOUBLE 
 60.0 
 120.0 
 15 
 73.49285714285713 
 14.688062357716753 
 65.0 
 70.0 
 74.0 
 15 
 6.67

In [37]:
SELECT DISTINCT ON (function_name)
    function_name,
    function_type,
    return_type,
    parameters,
    parameter_types,
    description
FROM duckdb_functions()
WHERE function_type = 'aggregate'
  AND function_name LIKE 's%'
ORDER BY function_name;



function_name 
 function_type 
 return_type 
 parameters 
 parameter_types 
 description 
 
 
 
 
 0 
 sem 
 aggregate 
 DOUBLE 
 [x] 
 [DOUBLE] 
 Returns the standard error of the mean 
 
 
 1 
 skewness 
 aggregate 
 DOUBLE 
 [x] 
 [DOUBLE] 
 Returns the skewness of all input values. 
 
 
 2 
 stddev 
 aggregate 
 DOUBLE 
 [x] 
 [DOUBLE] 
 Returns the sample standard deviation 
 
 
 3 
 stddev_pop 
 aggregate 
 DOUBLE 
 [x] 
 [DOUBLE] 
 Returns the population standard deviation. 
 
 
 4 
 stddev_samp 
 aggregate 
 DOUBLE 
 [x] 
 [DOUBLE] 
 Returns the sample standard deviation 
 
 
 5 
 string_agg 
 aggregate 
 VARCHAR 
 [str] 
 [ANY] 
 Concatenates the column string values with an ... 
 
 
 6 
 sum 
 aggregate 
 DECIMAL 
 [arg] 
 [DECIMAL] 
 Calculates the sum value for all tuples in arg. 
 
 
 7 
 sum_no_overflow 
 aggregate 
 HUGEINT 
 [arg] 
 [INTEGER] 
 Internal only. Calculates the sum value for al... 
 
 
 8 
 sumkahan 
 aggregate 
 DOUBLE 
 [arg] 
 [DOUBLE] 
 Calculates the sum using a more accurate float...

In [38]:
import pandas as pd

# Define a dictionary with DuckDB features, descriptions, and example usage
data = {
    "Feature": [
        "Parquet Files",
        "CSV Files",
        "JSON Files",
        "Window Functions",
        "Scalar Functions",
        "Aggregate Functions",
        "Python Integration",
        "SQLite Integration"
    ],
    "Description": [
        "Query Parquet files directly without pre-loading data.",
        "Read CSV files with automatic schema inference.",
        "Query JSON files including nested structures.",
        "Use advanced window functions like row_number, rank, etc.",
        "Transform data with a wide range of scalar functions.",
        "Summarize data using functions like avg, sum, count, etc.",
        "Integrate with Pandas DataFrames and run SQL on Python data.",
        "Directly query SQLite databases from DuckDB."
    ],
    "Example Usage": [
        "SELECT * FROM read_parquet('data/sample.parquet');",
        "SELECT * FROM read_csv_auto('data/sample.csv');",
        "SELECT * FROM read_json('data/sample.json');",
        "SELECT col, ROW_NUMBER() OVER (ORDER BY col) AS rn FROM my_table;",
        "SELECT ABS(-5);",
        "SELECT AVG(price) FROM oil_prices;",
        "import duckdb\nduckdb.query('SELECT * FROM df').to_df();",
        "SELECT * FROM sqlite_scan('db.sqlite', 'table');"
    ]
}

# Create the DataFrame
duckdb_features_df = pd.DataFrame(data)

# Display the DataFrame
duckdb_features_df

Feature 
 Description 
 Example Usage 
 
 
 
 
 0 
 Parquet Files 
 Query Parquet files directly without pre-loadi... 
 SELECT * FROM read_parquet('data/sample.parque... 
 
 
 1 
 CSV Files 
 Read CSV files with automatic schema inference. 
 SELECT * FROM read_csv_auto('data/sample.csv'); 
 
 
 2 
 JSON Files 
 Query JSON files including nested structures. 
 SELECT * FROM read_json('data/sample.json'); 
 
 
 3 
 Window Functions 
 Use advanced window functions like row_number,... 
 SELECT col, ROW_NUMBER() OVER (ORDER BY col) A... 
 
 
 4 
 Scalar Functions 
 Transform data with a wide range of scalar fun... 
 SELECT ABS(-5); 
 
 
 5 
 Aggregate Functions 
 Summarize data using functions like avg, sum, ... 
 SELECT AVG(price) FROM oil_prices; 
 
 
 6 
 Python Integration 
 Integrate with Pandas DataFrames and run SQL o... 
 import duckdb\nduckdb.query('SELECT * FROM df'... 
 
 
 7 
 SQLite Integration 
 Directly query SQLite databases from DuckDB. 
 SELECT * FROM sqlite_scan('db.sqlite', 'table');

# Sheet 2